In [ ]:
import os
import tensorflow as tf
from multiprocessing import cpu_count

# from keras.models import * 
# from keras.layers import *
# from keras.applications import *
# from keras.preprocessing.image import *
# from keras.utils.training_utils import multi_gpu_model
# from multiprocessing import cpu_count
# import tensorflow as tf

In [ ]:
nb_classes = 2
image_size = (299, 299)
input_shape= (299, 299, 3)

nb_cpus = cpu_count()//2
nb_gpus = 2

train_path = "/home/hdd0/Develop/immune/ext-data/data299/train/"
valid_path = "/home/hdd0/Develop/immune/ext-data/data299/valid/"

In [ ]:
with tf.device('/cpu:0'):
    input_tensor = tf.keras.Input(input_shape)
    x = tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)(input_tensor)

    base_model = tf.keras.applications.Xception(input_tensor=x, weights='imagenet', include_top=False)
    m_out = base_model.output
    p_out = tf.keras.layers.GlobalAveragePooling2D()(m_out)
    p_out = tf.keras.layers.Dropout(0.5)(p_out)
    predictions = tf.keras.layers.Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

if nb_gpus > 1:
    parallel_model = tf.keras.utils.multi_gpu_model(model, gpus=nb_gpus)
parallel_model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

# with tf.device('/cpu:0'):
#     input_tensor = Input(input_shape)
#     x = Lambda(xception.preprocess_input)(input_tensor)

#     base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
#     m_out = base_model.output
#     p_out = GlobalAveragePooling2D()(m_out)
#     p_out = Dropout(0.5)(p_out)
#     predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

#     for layer in base_model.layers:
#         layer.trainable = False

#     model = Model(inputs=base_model.input, outputs=predictions)

# if nb_gpus > 1:
#     parallel_model = multi_gpu_model(model, gpus=nb_gpus)
# parallel_model.compile(optimizer='Adadelta', 
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])
# model.summary()

In [ ]:
batch_size = 64
epochs = 1

In [ ]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator),
                             workers=nb_cpus, 
                             use_multiprocessing=True)

model.save_weights("Xception_first_train.h5")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import multi_gpu_model
import numpy as np

num_samples = 32
height = 224
width = 224
num_classes = 1000

with tf.device('/cpu:0'):
    model = Xception(weights=None,input_shape=(height, width, 3),classes=num_classes)

# Replicates the model on 2 GPUs.
# This assumes that your machine has 2 available GPUs.
parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Generate dummy data.
x = np.random.random((num_samples, height, width, 3)).astype(np.float32)

dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.batch(32)
dataset = dataset.repeat()

iterator = dataset.make_one_shot_iterator()
batch = iterator.get_next()

y = parallel_model(batch)

sess = tf.keras.backend.get_session()
while True:
    try:
        result = sess.run(y)
        print(result.shape)
    except tf.errors.OutOfRangeError:
        break

In [ ]:
# I replaced the last section of your code

iterator = dataset.make_one_shot_iterator()
batch = iterator.get_next()

y = parallel_model(batch)

sess = tf.keras.backend.get_session()
while True:
    try:
        result = sess.run(y)
        print(result.shape)
    except tf.errors.OutOfRangeError:
        break
        
# with

parallel_model.fit(dataset, steps_per_epoch=1000)